# Keras tutorial

In [ ]:
import random

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, Flatten
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

from utils import display_samples, take_test_samples_idxs

## MNIST Dataset

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

In [ ]:
x_train[0].shape

In [ ]:
display_samples(x_train, y_train, n_rows=5, n_cols=8, label=6)

## MLP from scratch

### lista di strati passati al costruttore

In [ ]:
x_train = x_train / 255

In [ ]:
class Model:
    def __init__(self):
        self.W_0 = tf.Variable(initial_value=np.random.normal(size=(784, 100))*1e-3)
        self.b_0 = tf.Variable(initial_value=np.zeros(100))
        self.W_1 = tf.Variable(initial_value=np.random.normal(size=(100, 10))*1e-2)
        self.b_1 = tf.Variable(initial_value=np.zeros(10))
        self.vars = [self.W_0, self.W_1, self.b_0, self.b_1]

    def __call__(self, x):
        x = x.reshape(x.shape[0], -1)
        x_1 = tf.math.maximum(x@self.W_0 + self.b_0, 0)
        x_2 = (x_1@self.W_1 + self.b_1)
        return tf.nn.softmax(x_2, axis=1)

In [ ]:
model = Model()
NUM_EPOCHS = 5
N_TRAIN_SET = x_train.shape[0]
BATCH_SIZE = 32
LR = 1e-2
pbar = tqdm(range(NUM_EPOCHS))
for epoch in pbar:
    for i in range(0, N_TRAIN_SET, BATCH_SIZE):
        x_batch = x_train[i:i+BATCH_SIZE]
        with tf.GradientTape() as tape:
            y_prob = model(x_batch)
            losses = tf.keras.losses.categorical_crossentropy(tf.one_hot(y_train[i:i+BATCH_SIZE], 10), y_prob)
            loss = tf.math.reduce_mean(losses)
            grads = tape.gradient(loss, model.vars)
            pbar.set_description(str(loss.numpy().item()))
        for var, grad in zip(model.vars, grads):
            var.assign_sub(grad * LR)

### predizione

In [ ]:
y_prob = model(x=x_test).numpy()

In [ ]:
y_hat = np.argmax(y_prob, axis=1)

### accuratezza

In [ ]:
np.sum(y_hat == y_test) / y_test.size

## esame della predizione

### predizioni corrette con maggiore confidenza

In [ ]:
idxs = take_test_samples_idxs(y_prob, y_test, is_correct=True, most_confident=True, k=5*4)
display_samples(x_test[idxs], y_test[idxs], y_hat[idxs], y_prob[idxs], n_rows=5, n_cols=4)

### predizioni corrette con minore confidenza

In [ ]:
idxs = take_test_samples_idxs(y_prob, y_test, is_correct=True, most_confident=False, k=5*4)
display_samples(x_test[idxs], y_test[idxs], y_hat[idxs], y_prob[idxs], n_rows=5, n_cols=4)

### predizioni errate con maggiore confidenza

In [ ]:
idxs = take_test_samples_idxs(y_prob, y_test, is_correct=False, most_confident=True, k=5*4)
display_samples(x_test[idxs], y_test[idxs], y_hat[idxs], y_prob[idxs], n_rows=5, n_cols=4)

### predizioni errate con minore confidenza

In [ ]:
idxs = take_test_samples_idxs(y_prob, y_test, is_correct=False, most_confident=False, k=5*4)
display_samples(x_test[idxs], y_test[idxs], y_hat[idxs], y_prob[idxs], n_rows=5, n_cols=4)